In [1]:
import pandas as pd
import pathlib
import numpy as np

In [2]:
train_path = pathlib.Path("/home/starke88/data/haicu/covid_data_challenge_2021/trainSet.txt")
df_train = pd.read_csv(train_path)
df_train.set_index("PatientID", inplace=True)
df_train.drop(["ImageFile", "Hospital"], axis=1, inplace=True)
df_train["Prognosis"] = (df_train["Prognosis"] == "SEVERE").astype(np.uint8)

# one-hot encode the hospital
#foo = pd.get_dummies(df_train["Hospital"], prefix="Hospital")
#df_train = pd.concat([df_train, foo], axis=1)
#df_train = df_train.drop(["Hospital"], axis=1)


df_train

,Age,Sex,Temp_C,Cough,DifficultyInBreathing,WBC,CRP,Fibrinogen,LDH,Ddimer,Ox_percentage,PaO2,SaO2,pH,CardiovascularDisease,RespiratoryFailure,Prognosis
PatientID,,,,,,,,,,,,,,,,,
P_131,35.913889,0,39.3,1.0,0.0,5.76,43.40,651.0,387.0,157.0,94.0,NaN,NaN,NaN,0.0,NaN,0
P_132,57.266667,0,37.0,0.0,0.0,11.48,64.00,700.0,338.0,601.0,94.0,75.0,96.9,7.42,0.0,NaN,0
P_195,79.263889,0,37.8,1.0,0.0,6.21,115.30,698.0,356.0,448.0,94.0,63.0,94.6,7.39,1.0,NaN,1
P_193,82.000000,0,38.0,1.0,0.0,7.28,149.30,513.0,482.0,NaN,97.0,68.0,96.3,7.46,0.0,NaN,1
P_140,60.791667,1,37.0,1.0,0.0,6.37,20.70,NaN,NaN,210.0,93.0,NaN,97.3,NaN,0.0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
P_1_12,51.000000,0,NaN,0.0,1.0,14.30,22.79,550.0,368.0,5027.0,NaN,NaN,NaN,NaN,0.0,0.0,1
P_1_8,57.000000,0,NaN,1.0,0.0,5.10,8.93,757.0,451.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1
P_1_10,38.000000,0,NaN,0.0,1.0,7.30,0.23,NaN,NaN,1073.0,NaN,NaN,NaN,NaN,0.0,0.0,0


In [3]:
# this is how many complete samples we have
df_train.dropna().shape

(6, 17)

In [6]:
# how many samples are missing for individual columns
discard = []
discard_thresh = 200
for c in df_train.columns:
    n_missing = df_train[c].isna().sum()
    print(c, "\t\t\t", n_missing, "/", len(df_train))
    if n_missing > discard_thresh:
        discard.append(c)
        
print(f"\nGoing to discard features with more than {discard_thresh} nans:", discard)

Age 			 1 / 863
Sex 			 0 / 863
Temp_C 			 154 / 863
Cough 			 5 / 863
DifficultyInBreathing 			 4 / 863
WBC 			 9 / 863
CRP 			 33 / 863
Fibrinogen 			 591 / 863
LDH 			 136 / 863
Ddimer 			 621 / 863
Ox_percentage 			 243 / 863
PaO2 			 170 / 863
SaO2 			 583 / 863
pH 			 207 / 863
CardiovascularDisease 			 19 / 863
RespiratoryFailure 			 159 / 863
Prognosis 			 0 / 863

Going to discard features with more than 200 nans: ['Fibrinogen', 'Ddimer', 'Ox_percentage', 'SaO2', 'pH']


In [7]:
df_reduced = df_train.drop(discard, axis=1)
df_reduced

,Age,Sex,Temp_C,Cough,DifficultyInBreathing,WBC,CRP,LDH,PaO2,CardiovascularDisease,RespiratoryFailure,Prognosis
PatientID,,,,,,,,,,,,
P_131,35.913889,0,39.3,1.0,0.0,5.76,43.40,387.0,NaN,0.0,NaN,0
P_132,57.266667,0,37.0,0.0,0.0,11.48,64.00,338.0,75.0,0.0,NaN,0
P_195,79.263889,0,37.8,1.0,0.0,6.21,115.30,356.0,63.0,1.0,NaN,1
P_193,82.000000,0,38.0,1.0,0.0,7.28,149.30,482.0,68.0,0.0,NaN,1
P_140,60.791667,1,37.0,1.0,0.0,6.37,20.70,NaN,NaN,0.0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
P_1_12,51.000000,0,NaN,0.0,1.0,14.30,22.79,368.0,NaN,0.0,0.0,1
P_1_8,57.000000,0,NaN,1.0,0.0,5.10,8.93,451.0,NaN,0.0,0.0,1
P_1_10,38.000000,0,NaN,0.0,1.0,7.30,0.23,NaN,NaN,0.0,0.0,0


In [8]:
df_reduced = df_reduced.dropna()
df_reduced.describe()

,Age,Sex,Temp_C,Cough,DifficultyInBreathing,WBC,CRP,LDH,PaO2,CardiovascularDisease,RespiratoryFailure,Prognosis
count,417.000000,417.000000,417.000000,417.000000,417.000000,417.000000,417.000000,417.000000,417.000000,417.000000,417.000000,417.000000
mean,65.167866,0.338129,37.553717,0.522782,0.489209,6.953189,32.663094,370.949640,73.613669,0.294964,0.014388,0.482014
std,14.489739,0.473641,0.968887,0.500081,0.500484,3.279598,54.263935,229.063855,26.501603,0.456574,0.119229,0.500277
min,22.000000,0.000000,35.400000,0.000000,0.000000,0.200000,0.010000,115.000000,23.000000,0.000000,0.000000,0.000000
25%,56.000000,0.000000,36.800000,0.000000,0.000000,4.700000,5.360000,247.000000,60.000000,0.000000,0.000000,0.000000
50%,65.000000,0.000000,37.700000,1.000000,0.000000,6.300000,14.420000,318.000000,70.000000,0.000000,0.000000,0.000000
75%,78.000000,1.000000,38.200000,1.000000,1.000000,8.460000,33.960000,422.000000,80.000000,1.000000,0.000000,1.000000
max,96.000000,1.000000,40.000000,1.000000,1.000000,26.500000,413.000000,2578.000000,285.000000,1.000000,1.000000,1.000000


In [9]:
label_col = "Prognosis"

y = df_reduced[[label_col]]
X_df = df_reduced[[c for c in df_reduced.columns if c not in [label_col]]]

X_df

,Age,Sex,Temp_C,Cough,DifficultyInBreathing,WBC,CRP,LDH,PaO2,CardiovascularDisease,RespiratoryFailure
PatientID,,,,,,,,,,,
P_302,56.0,0,38.3,0.0,1.0,7.85,9.00,159.0,73.2,0.0,0.0
P_301,61.0,0,38.0,1.0,1.0,3.57,57.40,309.0,57.0,0.0,0.0
P_268,64.0,0,36.0,1.0,0.0,5.26,41.90,299.0,72.2,1.0,0.0
P_282,54.0,0,36.6,1.0,0.0,7.01,67.90,458.0,61.0,0.0,0.0
P_300,44.0,0,37.7,0.0,0.0,9.18,42.70,243.0,23.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
P_1_77,61.0,1,36.0,1.0,0.0,3.60,0.07,219.0,86.2,0.0,0.0
P_1_110,56.0,0,36.5,1.0,0.0,7.00,13.51,411.0,75.8,0.0,0.0
P_1_126,57.0,0,37.0,1.0,1.0,4.75,2.47,383.0,61.0,0.0,0.0


In [10]:
picked_features = X_df.columns

#picked_features = [
#    "LDH",
#    "Age",
#    "PaO2",
#    "CRP",
#    "WBC",
#    "pH"
#]


X = X_df[picked_features]


In [24]:
# now also load the test data for which maksim imputed the values

test_path = pathlib.Path("/home/starke88/data/haicu/covid_data_challenge_2021/testSet_filled.txt")
df_test = pd.read_csv(test_path)
df_test.set_index("PatientID", inplace=True)
df_test.drop(["ImageFile", "Unnamed: 0", "Prognosis"], axis=1, inplace=True)
# get rid of the features we dumped from the training set as well
df_test.drop([c for c in df_test.columns if c not in df_reduced.columns], axis=1, inplace=True)
# one-hot encode the hospital
#foo = pd.get_dummies(df_test["Hospital"], prefix="Hospital")
#df_test = pd.concat([df_test, foo], axis=1)
#df_test = df_test.drop(["Hospital"], axis=1)
assert sorted(df_test.columns) == sorted(X.columns)


X_test = df_test

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [26]:
train, valid = train_test_split(X_df.index.to_numpy(), test_size=.25, stratify=y[label_col], random_state=52)
print(len(train), len(valid))

312 105


In [27]:
print(valid)
pd.DataFrame(valid).to_csv("/home/starke88/git/covid_data_challenge/clinical_model/valid_ids.csv", index=False, header=None)

['P_384' 'P_417' 'P_551' 'P_479' 'P_555' 'P_376' 'P_1_26' 'P_379' 'P_441'
 'P_450' 'P_760' 'P_756' 'P_435' 'P_442' 'P_766' 'P_695' 'P_537' 'P_687'
 'P_467' 'P_1_126' 'P_1_75' 'P_653' 'P_1_149' 'P_411' 'P_358' 'P_740'
 'P_1_81' 'P_634' 'P_616' 'P_382' 'P_342' 'P_519' 'P_476' 'P_639' 'P_759'
 'P_350' 'P_484' 'P_635' 'P_507' 'P_439' 'P_798' 'P_598' 'P_414' 'P_583'
 'P_590' 'P_485' 'P_511' 'P_426' 'P_304' 'P_842' 'P_797' 'P_837' 'P_314'
 'P_572' 'P_641' 'P_803' 'P_1_55' 'P_702' 'P_691' 'P_518' 'P_1_95' 'P_521'
 'P_589' 'P_840' 'P_1_131' 'P_772' 'P_368' 'P_292' 'P_750' 'P_274'
 'P_1_146' 'P_650' 'P_505' 'P_733' 'P_623' 'P_697' 'P_620' 'P_1_140'
 'P_609' 'P_808' 'P_526' 'P_383' 'P_595' 'P_587' 'P_580' 'P_502' 'P_1_22'
 'P_324' 'P_1_72' 'P_725' 'P_336' 'P_491' 'P_781' 'P_407' 'P_546' 'P_645'
 'P_425' 'P_735' 'P_445' 'P_812' 'P_385' 'P_1_66' 'P_743' 'P_560' 'P_633']


In [28]:
train[:4]

array(['P_428', 'P_399', 'P_499', 'P_282'], dtype=object)

In [29]:
mask_train = X.index.isin(train)
feat_names = X.columns

X_train = X[mask_train]
y_train = y[mask_train]

X_valid = X[~mask_train]
y_valid = y[~mask_train]

print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)

(312, 11) (312, 1)
(105, 11) (105, 1)


In [30]:
X_train.dtypes

Age                      float64
Sex                        int64
Temp_C                   float64
Cough                    float64
DifficultyInBreathing    float64
WBC                      float64
CRP                      float64
LDH                      float64
PaO2                     float64
CardiovascularDisease    float64
RespiratoryFailure       float64
dtype: object

In [31]:
def get_patient_ids(df):
    return df.index.to_numpy().squeeze().tolist()

scaler = MinMaxScaler()
#scaler = StandardScaler()

ids_train = get_patient_ids(X_train)
X_train = scaler.fit_transform(X_train.to_numpy())
y_train = y_train.to_numpy().squeeze()

ids_valid = get_patient_ids(X_valid)
X_valid = scaler.transform(X_valid.to_numpy())
y_valid = y_valid.to_numpy().squeeze()

ids_test = get_patient_ids(X_test)
X_test = scaler.transform(X_test.to_numpy())

In [32]:
len(ids_train), len(ids_valid), len(ids_test)

(312, 105, 120)

In [33]:
print(X_train.shape, y_train.shape, len(ids_train))
print(X_valid.shape, y_valid.shape, len(ids_valid))
print(X_test.shape, len(ids_test))

(312, 11) (312,) 312
(105, 11) (105,) 105
(120, 11) 120


In [34]:
## simple logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, confusion_matrix

#model = RandomForestClassifier() 
model = LogisticRegression(penalty="none", solver="lbfgs")

model.fit(X_train, y_train)
if hasattr(model, "coef_"):
    coeffs = dict(zip(feat_names, model.coef_[0]))
    print(coeffs)
    print("\nsignificant coeffs", {k:v for k,v in coeffs.items() if np.abs(v) > .01})
print()

def evaluate_model(model, X, y):
    pred_y = model.predict(X)

    print("-Classification metrics\n", classification_report(y, pred_y))
    print()
    print("-Confusion matrix\n", confusion_matrix(y, pred_y))
    print()
    print("-AUC", roc_auc_score(y, pred_y))
    

def make_prediction_df(model, X, patient_ids, method_name):
    cls_1_index = np.where(model.classes_ == 1)[0][0]
    pred_score = model.predict_proba(X)[:, cls_1_index]
    return pd.DataFrame({"PatientID": patient_ids, f"prediction_{method_name}": pred_score})
    
    
print("Training performance\n=====")
evaluate_model(model, X_train, y_train)

print("Validation performance\n=====")
evaluate_model(model, X_valid, y_valid)

{'Age': 4.438642485192933, 'Sex': -0.5380532156403621, 'Temp_C': 0.9903044054257356, 'Cough': -0.09706348594397517, 'DifficultyInBreathing': 0.7928339789391249, 'WBC': 0.9729638366525475, 'CRP': 1.5618131479569666, 'LDH': 16.805483182146066, 'PaO2': -0.1568112199646922, 'CardiovascularDisease': -0.2591747799837606, 'RespiratoryFailure': -0.7772138167663587}

significant coeffs {'Age': 4.438642485192933, 'Sex': -0.5380532156403621, 'Temp_C': 0.9903044054257356, 'Cough': -0.09706348594397517, 'DifficultyInBreathing': 0.7928339789391249, 'WBC': 0.9729638366525475, 'CRP': 1.5618131479569666, 'LDH': 16.805483182146066, 'PaO2': -0.1568112199646922, 'CardiovascularDisease': -0.2591747799837606, 'RespiratoryFailure': -0.7772138167663587}

Training performance
=====
-Classification metrics
               precision    recall  f1-score   support

           0       0.77      0.79      0.78       162
           1       0.77      0.74      0.75       150

    accuracy                           0.77

In [38]:
model_name = "Clinical-logistic-regression-nopenalty-lbfgs-trainedOn312Patients"
pred_train = make_prediction_df(model, X_train, patient_ids=ids_train, 
                   method_name=model_name)

pred_valid = make_prediction_df(model, X_valid, patient_ids=ids_valid, 
                   method_name=model_name)

pred_test = make_prediction_df(model, X_test, patient_ids=ids_test, 
                   method_name=model_name)
pred_test.to_csv(f"/home/starke88/git/covid_data_challenge/clinical_model/predictions_{model_name}.csv", index=False)

In [39]:
if hasattr(model, "feature_importances_"):
    imp = dict(zip(feat_names, model.feature_importances_))
    for k, v in imp.items():
        print(k,v)

In [43]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, KFold

# l1 or l2
logistic_pipe = Pipeline([
    #("scaling", MinMaxScaler()),
    ("fit", LogisticRegression(solver="liblinear"))])

logistic_grid = {
    #"scaling": ["passthrough", MinMaxScaler(), StandardScaler()],
    "fit__penalty": ["l1", "l2"],
    "fit__class_weight": [None, "balanced"]}


searcher = GridSearchCV(
            estimator=logistic_pipe,
            param_grid=logistic_grid,
            cv=KFold(n_splits=5, shuffle=False),
            #scoring="neg_mean_absolute_error",
            verbose=1)
searcher.fit(X_train, y_train)
print(": grid-search best parameters", searcher.best_params_)
print(": grid-search best score", searcher.best_score_)
print()
print("\nTraining performance of best model\n")
evaluate_model(searcher.best_estimator_, X_train, y_train)
print("\nValidation performance of best model\n")
evaluate_model(searcher.best_estimator_, X_valid, y_valid)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
: grid-search best parameters {'fit__class_weight': 'balanced', 'fit__penalty': 'l1'}
: grid-search best score 0.7469022017409115


Training performance of best model

-Classification metrics
               precision    recall  f1-score   support

           0       0.76      0.78      0.77       162
           1       0.76      0.73      0.74       150

    accuracy                           0.76       312
   macro avg       0.76      0.76      0.76       312
weighted avg       0.76      0.76      0.76       312


-Confusion matrix
 [[127  35]
 [ 41 109]]

-AUC 0.7553086419753087

Validation performance of best model

-Classification metrics
               precision    recall  f1-score   support

           0       0.75      0.78      0.76        54
           1       0.76      0.73      0.74        51

    accuracy                           0.75       105
   macro avg       0.75      0.75      0.75       105
weighted avg    

In [44]:
model_name = "Clinical-logistic-regression-gridsearchedParams-l1penalty-liblinear-trainedOn312Patients"
model = searcher.best_estimator_

pred_train = make_prediction_df(model, X_train, patient_ids=ids_train, 
                   method_name=model_name)

pred_valid = make_prediction_df(model, X_valid, patient_ids=ids_valid, 
                   method_name=model_name)

pred_test = make_prediction_df(model, X_test, patient_ids=ids_test, 
                   method_name=model_name)

pred_test.to_csv(f"/home/starke88/git/covid_data_challenge/clinical_model/predictions_{model_name}.csv", index=False)

In [45]:
pred_test

,PatientID,prediction_Clinical-logistic-regression-gridsearchedParams-l1penalty-liblinear-trainedOn312Patients
0,P_102,0.320189
1,P_117,0.147684
2,P_16,0.250667
3,P_118,0.508742
4,P_114,0.412056
...,...,...
115,P_88,0.354125
116,P_92,0.139153
117,P_86,0.707364
118,P_9,0.597842


In [46]:
rf_pipe = Pipeline([
    #("scaling", MinMaxScaler()),
    ("fit", RandomForestClassifier(n_estimators=100,
                                  # max_depth=5,
                                  # max_leaf_nodes=20,
                                  # min_impurity_decrease=.1
                                  random_state=42,
                                  ))
])
rf_grid = {
    #"scaling": [MinMaxScaler(), StandardScaler()],
    "fit__n_estimators": [5, 10, 20, 100, 200],
    "fit__max_depth": [None, 5, 10, 20, 50],
    "fit__max_leaf_nodes": [None, 20, 50, 100],
    "fit__min_impurity_decrease": [0, 0.1, .5, 1]    }

searcher = GridSearchCV(
            estimator=rf_pipe,
            param_grid=rf_grid,
            cv=KFold(n_splits=5, shuffle=False),
            #scoring="neg_mean_absolute_error",
            verbose=1)
searcher.fit(X_train, y_train)

print(": grid-search best parameters", searcher.best_params_)
print(": grid-search best score", searcher.best_score_)
print()
print("\nTraining performance of best model\n")
evaluate_model(searcher.best_estimator_, X_train, y_train)
print("\nValidation performance of best model\n")
evaluate_model(searcher.best_estimator_, X_valid, y_valid)

Fitting 5 folds for each of 400 candidates, totalling 2000 fits
: grid-search best parameters {'fit__max_depth': 5, 'fit__max_leaf_nodes': 20, 'fit__min_impurity_decrease': 0, 'fit__n_estimators': 10}
: grid-search best score 0.7596006144393241


Training performance of best model

-Classification metrics
               precision    recall  f1-score   support

           0       0.89      0.84      0.87       162
           1       0.84      0.89      0.86       150

    accuracy                           0.87       312
   macro avg       0.87      0.87      0.87       312
weighted avg       0.87      0.87      0.87       312


-Confusion matrix
 [[136  26]
 [ 16 134]]

-AUC 0.8664197530864197

Validation performance of best model

-Classification metrics
               precision    recall  f1-score   support

           0       0.81      0.72      0.76        54
           1       0.74      0.82      0.78        51

    accuracy                           0.77       105
   macro avg   

In [47]:
model_name = "Clinical-randomforest-10estimators-trainedOn312Patients"
model = searcher.best_estimator_

pred_train = make_prediction_df(model, X_train, patient_ids=ids_train, 
                   method_name=model_name)

pred_valid = make_prediction_df(model, X_valid, patient_ids=ids_valid, 
                   method_name=model_name)
pred_test = make_prediction_df(model, X_test, patient_ids=ids_test, 
                   method_name=model_name)
pred_test.to_csv(f"/home/starke88/git/covid_data_challenge/clinical_model/predictions_{model_name}.csv", index=False)